이번 글에서는 [벡터 연산](http://wikibootup.github.io/sicp/2-2-vector-operations.html)에 이어서 고차함수를(`map`과 같은) 이용하여 두가지 알고리즘을 구현하겠습니다.

### 호너의 규칙

$a_nx^n + a_{n-1}x^{n-1} + ... + a_1x + a_0$

위의 다항식은 호너의 규칙에 따라 아래와 같이 바꿀 수 있습니다.

$(...(a_nx + a_{n-1})x + ... + a_1)x + a_0$

(특이한 공식을 사용한 것은 아니고 공통 인수인 $x$로 묶어둔 것입니다.) 

이것을 함수적으로 바꿔보면,

```HornerEval x higherTerms
-> FirstTerm + x * HornerEval x (higherTerms+1)
-> FirstTerm + x * (SecondTerm + x * HornerEval x (higherTerms+1))
-> ...
```

`map`과 `accumulate` 함수를 이용해 이런 생각을 그대로 옮겨보면,

In [8]:
hornerEval :: (Num a, Foldable t) => a -> t a -> a
hornerEval x coeffSeqs = 
    foldl1  (\thisCoeff higherTerms -> x * thisCoeff + higherTerms)
            coeffSeqs

* 이전에 `Python`이나 `Scheme`(Git repo 상에 있는)에서는 `accumulate`라는 함수를 이용해서 리스트의 각 항에 필요한 연산을 적용했습니다. 여기서는 같은 기능인데 이름만 다른 `fold`라는 함수를 이용하였습니다.

아래의 경우로 테스트해보겠습니다.

1. 계수 고정 : $2^{15} + 2^{14} + ... + 2 + 1 = 2^{16}-1$

2. 홀수 항만 : $2^{15} + 2^{13} + ... + 2^3 + 2^1 = ?$

3. 짝수 항만 : $2^{14} + 2^{12} + ... + 2^2 = ?$

4. 계수 변화 : $1*2^{15} + 2*2^{14} + ... + 16 = ?$


* 편의상 $x=2$로 고정하겠습니다.

In [285]:
hornerEval 2 (replicate 16 1) == 2^16-1

True

In [225]:
hornerEval 2 (concat (replicate 7 [1,0]) ++ [0]) == 
    foldl (\acc x -> 2^x + acc) 0 [x*2 | x <- [1..7]] 

True

In [226]:
hornerEval 2 (concat (replicate 8 [0,1]) ++ [0]) == 
    foldl (\acc x -> 2^x + acc) 0 [x*2-1 | x <- [1..8]] 

True

In [193]:
hornerEval 2 [1..16] == 
    sum (zipWith (*) (map (\ x -> 2^x) (reverse [0 .. 15])) 
        [1..16])

Not in scope: ‘hornerEval’

### 퀸 퍼즐

퀸 퍼즐은 체스판에 퀸을 배열하는 문제입니다. 규칙은 다음과 같습니다.

1. 세로줄마다 퀸은 더도 덜도 말고 딱 1개 위치해야 한다.
2. 각 퀸은 서로 공격할 수 없는 위치에 있어야 한다.

( * 구현 방식을 이해하는 데 있어서 다음 링크에 나와 있는 시뮬레이션이 많은 도움이 되었습니다. https://www.cs.usfca.edu/~galles/visualization/RecQueens.html )

문제가 다소 복잡하므로 부분적인 기능부터 완전히 만들도록 하겠습니다.

먼저, 체스판 위에 새로운 퀸을 두려 할 때 그 위치에 놓을 수 있는지를 확인하는 기능을 구현하겠습니다.

```
이름 : isSafe
인자 : tryPos, posQueens
    tryPos : 새로운 퀸을 두려는 위치, 데이터 타입은 (a,a)
    posQueens : 이미 위치한 퀸의 위치, 데이터 타입은 [(a,a)]

```
퀸은 1. 수직, 2. 수평, 3. 대각선이면 어디든 갈 수 있습니다. 반대로 말하면, 방금 말한 세가지만 확인하면 안전한 위치인지 확인할 수 있습니다.

예를 들어, $0 < b <= a < n$인 $(a,b)$의 위치에 퀸을 두려한다고 하면, 다음 위치에 퀸이 없어야 합니다.

```
1. 수직 : (0,b), (1,b), ... , (n-1,b)
2. 수평 : (a,0), (a,1), ... , (a, n-1)
3. 대각선 : (a-b, 0), (a-b+1, 1), ... , (n-1, n-(1+(a-b))
```

위를 통해, 수직과 수평에 대해서 확인을 할 때는 0부터 n-1의 위치까지 확인하면 됨을 알 수 있습니다.

하지만 여기서는 그럴 필요 없이,

1. 수직에 대해서는 같은 열Column에 위치한 퀸이 있는지 확인
2. 수평에 대해서는 같은 행Row에 위치한 퀸이 있는지 확인

위와 같이 확인하도록 하겠습니다.

비슷하게 대각선에 대해서는 좌표를 이동하며 비교하지 않고 아래의 방법으로 확인하겠습니다.

1. 각 퀸의 위치 마다, 그리고 새로 두려는 위치에 `(a-b, 0)` 연산을 수행 (이 때, $0 < b <= a < n$)
2. 같은 경우가 있는지 확인


* 이렇게 구현하면 체스판의 크기(n)와 무관하게 수직,수평,대각선을 확인할 수 있습니다. 따라서 함수의 기능을 파악하는게 한결 간단해집니다.

구현하면,

In [6]:
isSafe :: (Num a, Ord a) => (a, a) -> [(a, a)] -> Bool
isSafe tryPos posQueens = isSafeRow && isSafeCol && isSafeCross
    where
        isSafeRow = snd tryPos `notElem` map snd posQueens
        isSafeCol = fst tryPos `notElem` map fst posQueens
        isSafeCross = crossStartingPos tryPos `notElem` map crossStartingPos posQueens
            where
                crossStartingPos posQueens
                    | uncurry (<=) posQueens = (0, snd posQueens - fst posQueens)
                    | otherwise = (uncurry (-) posQueens, 0)

8퀸 퍼즐의 답 중 하나를 이용해 테스트 해보면,

In [7]:
posQueens1 = [(1,6),(7,5),(5,4),(0,3),(2,2),(4,1),(6,0)]
tryPosList = map (\x -> (x,7)) [0..7]
tryPosList

[(0,7),(1,7),(2,7),(3,7),(4,7),(5,7),(6,7),(7,7)]

In [8]:
filter (`isSafe` posQueens1) tryPosList

[(3,7)]

`queenList1`의 배열에서 7번째 열Column에 퀸을 둘 수 있는 위치는 (3,7) 하나입니다. (답과 같습니다)

이제 위의 방법을 이용해 열Column마다 퀸을 둘 수 있는 위치를 걸러내도록 하겠습니다.

In [9]:
n = 7 {--# temp value for board size #--}
safeColPosList col = filter (`isSafe` posQueens1) (map (\row -> (row, col)) [0..n])
safeColPosList 7

[(3,7)]

In [258]:
| safeColPosList == [] = _queenPuzzle (head (tail safeColPosList)) col

| _queenPuzzle (head safeColPosList ++ queenPosList) col+1

queenPuzzle n
_queenPuzzle queenPosList col



Parse error (line 1, column 1): parse error on input ‘|’

Parse error (line 3, column 1): parse error on input ‘|’

In [12]:
[] == null

Couldn't match expected type ‘[()]’ with actual type ‘t0 a0 -> Bool’
Probable cause: ‘null’ is applied to too few arguments
In the second argument of ‘(==)’, namely ‘null’
In the expression: [] == null

백트래킹은 메아리와 비슷한 원리

In [20]:
queenPuzzle n = _queenPuzzle [] 0 0
    where
        _queenPuzzle posQueens row col
            _queenPuzzle (tail posQueens) (row+1) col
            | null nextSafePos = _queenPuzzle (tail posQueens) (row+1) col
            | otherwise = nextSafePos:safePos 0 posQueens 
            | otherwise = nextSafePos 0 (col+1)
            
            _queenPuzzle (tail posQueens) (row+1) col
            | otherwise = _queenPuzzle (row,col):posQueens row (col+1)
            where
                safePos posQueens row col
                    | col == n = posQueens
                    | row == n = []
                    | isSafePos = (row,col)
                    | otherwise = safePos posQueens (row+1) col
                nextSafePos = safePos 0 (col+1)
                    where
                        isSafePos = isSafe (row,col) posQueens

Parse error (line 5, column 13): parse error on input ‘|’

In [1]:
abc 1

Not in scope: ‘abc’
Perhaps you meant ‘abs’ (imported from Prelude)

In [ ]:
                    
            _queenPuzzleCol 0 col posQueens
        
        _queenPuzzleCol posQueens
            | col == (n-1) = posQueens
            | not _queenPuzzleRow row posQueens = _queenPuzzleRow (row+1) posQueens
        _queenPuzzleRow row posQueens
                
                
            | col == (n-1) = posQueens
            | row == (n-1) = posQueens
            | isSafe (row,col) posQueens = _queenPuzzle ((row,col):posQueens) 0 (col+1)
            | otherwise = _queenPuzzle posQueens (row+1) col

In [306]:
queenPuzzle 4

[(1,2),(2,1),(0,0)]

In [295]:
queenPuzzle n = _queenPuzzle [] 0 0
    where
        _queenPuzzle queenPosList row col
            | col == n = queenPosList
            | row == n = []
            | isSafe (row,col) queenPosList = _queenPuzzle ((row,col):queenPosList) 0 (col+1)
            | otherwise = _queenPuzzle queenPosList (row+1) col

In [296]:
queenPuzzle 8

[]

In [ ]:
_queenPuzzle adjustPos prevRowPos prevColPos

In [ ]:
            | col == (n-1) = True

In [ ]:
            | otherwise = []
                where
                    adjustPos = (prevRowPos+1, prevColPos):tail queenPosList
                        where
                            prevRowPos = fst (head queenPosList)
                            prevColPos = snd (head queenPosList)

In [ ]:
            | isSafe (row,col) = _queenPuzzle ((row,col):queenPosList) 0 (col+1)


In [ ]:
queenPuzzle n = _queenPuzzle [] 0 0
    where
        _queenPuzzle queenPosList row col
            | row == (n-1) = _queenPuzzle ((fst (head queenPosList)+1) snd (head queenPosList)):tail queenPosList (fst (head queenPosList)+1) (snd (head queenPosList)-1)
            | isSafe (row,col) = _queenPuzzle ((row,col):queenPosList) 0 (col+1)
            | otherwise = queenPosList (row+1) col



In [208]:
where
    PrevRowPos = fst (head queenPosList)
    PrevColPos = snd (head queenPosList)

True

In [207]:
(1,2):[(3,4)]

[(1,2),(3,4)]

In [213]:
fst $ head [(2,4)]

2

In [206]:
head $ tail [1,2,3]

2

In [71]:
when combine position, use :
extract one position, use tail

Parse error (line 1, column 22): parse error on input ‘,’

Parse error (line 2, column 21): parse error on input ‘,’

필터로 후보들을 넣어두자

In [ ]:
queens2 = []

In [87]:
filter (`isSafe` []) tryPositions

[(1,7),(2,7),(3,7),(4,7),(5,7),(6,7),(7,7)]

In [88]:
head $ filter (`isSafe` []) map (\row-> $ row,col)

(1,7)

In [139]:
flatmap (\col -> (map (\row -> (row,col)) [0..7])) [0..7]

[(0,0),(1,0),(2,0),(3,0),(4,0),(5,0),(6,0),(7,0),(0,1),(1,1),(2,1),(3,1),(4,1),(5,1),(6,1),(7,1),(0,2),(1,2),(2,2),(3,2),(4,2),(5,2),(6,2),(7,2),(0,3),(1,3),(2,3),(3,3),(4,3),(5,3),(6,3),(7,3),(0,4),(1,4),(2,4),(3,4),(4,4),(5,4),(6,4),(7,4),(0,5),(1,5),(2,5),(3,5),(4,5),(5,5),(6,5),(7,5),(0,6),(1,6),(2,6),(3,6),(4,6),(5,6),(6,6),(7,6),(0,7),(1,7),(2,7),(3,7),(4,7),(5,7),(6,7),(7,7)]

In [131]:
flatmap (\col -> (map (\row -> (row,col)) [0..7])) [0..7]

[(0,0),(1,0),(2,0),(3,0),(4,0),(5,0),(6,0),(7,0),(0,1),(1,1),(2,1),(3,1),(4,1),(5,1),(6,1),(7,1),(0,2),(1,2),(2,2),(3,2),(4,2),(5,2),(6,2),(7,2),(0,3),(1,3),(2,3),(3,3),(4,3),(5,3),(6,3),(7,3),(0,4),(1,4),(2,4),(3,4),(4,4),(5,4),(6,4),(7,4),(0,5),(1,5),(2,5),(3,5),(4,5),(5,5),(6,5),(7,5),(0,6),(1,6),(2,6),(3,6),(4,6),(5,6),(6,6),(7,6),(0,7),(1,7),(2,7),(3,7),(4,7),(5,7),(6,7),(7,7)]

In [159]:
filter (\x -> isSafe (x,0) []) [0..7]

[0,1,2,3,4,5,6,7]

In [150]:
[] ++ [1]

[1]

In [165]:
abc col = filter (`isSafe` queens1) (map (\row -> (row, col)) [0..7])

In [167]:
abc 7

[(3,7)]

In [144]:
flatmap :: (a1 -> [a]) -> [a1] -> [a]
flatmap f s = foldl1 (++) (map f s)

In [5]:
flatmap (\i -> map (\j -> [i,j]) [1..3]) [1..3]

[[1,1],[1,2],[1,3],[2,1],[2,2],[2,3],[3,1],[3,2],[3,3]]

In [6]:
:t flatmap

flatmap :: forall a1 a. (a1 -> [a]) -> [a1] -> [a]

8 Queens good reference
https://www.cs.usfca.edu/~galles/visualization/RecQueens.html

```
queen idea
1. 리스트에 위치 입력
맨 처음은 (0,0)에 퀸을 찍는다. 이 좌표를 리스트로 만들고 재귀 함수에 인자로 집어 넣는다. (이 때, 늦게 들어오는 퀸 위치를 리스트에 처음에 삽입한다)

2. 위치 탐색
그 다음은 (0,1)좌표에 퀸을 둘 수 있는지 인자로 넘어온 리스트를 통해 확인한다(직선과 대각선에서 만나지 않도록)

3. 백트래킹
(n,1)까지 그런 식으로 진행해서 가능한 위치가 존재하지 않는다면 이전에 지정한 리스트를 삭제하고 column은 -1 한 뒤 다시 시작한다.

4. 가능한 경우의 수 모두 수집


준비
1. (0 0)~(n-1 n-1)을 출력하는 함수를 만든다
2. setPosition
3. isSafe
```

In [1]:
isSafeVertical col colPosQueen = _isSafeVertical (0, col)
    where 
        _isSafeVertical (a,b)
            | a <= 7 = (a,b) /= colPosQueen && _isSafeVertical (a+1,b)
            | otherwise = True

In [4]:
:t isSafe

isSafe :: forall a. (Num a, Ord a) => (a, a) -> [(a, a)] -> Bool